# Training a multivariate discrimination

Below we prepare a simple setup to train a multivariate discriminator to separate the signal from the background.
It's meant just an appetizer for you to explore. After you go through it it may be a good occasion to put in practice what you learned in the short exercise about Machine Learning!

In [ ]:
#!pip install --user mplhep #uncomment if loading of mplhep fails

import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import mplhep
from matplotlib.colors import LogNorm

In [ ]:
#global configurarables (check the Data-Inspection notebook for consistency)
proton_selection = "MultiRP"
PATH='/eos/user/c/cmsdas/short-exercises/pps-protons-tutorial/data'

#eras=['B','C1','D','F1'] #uncomment to use all data
eras=['B']

## Collect the data and simulation

The following cell is used to load the training data to memory. Further pre-selection will be applied below.

The data which we are going to use to model the background has been pre-processed with an event mixing technique.
For each event the reconstructed protons have been substituted with protons from another event,
guaranteeing that there is no correlation between the central event and the one reconstucted in the PPS.

In [ ]:
def GetData(flist,chunk_size=None):
    
    """ 
    opens a summary file or list of summary files and convert them to a pandas dataframe 
    if given the chunk_size will be used to collect events in chunks of this size
    """
    
    flist=flist if isinstance(flist,list) else [flist]
    
    df,df_counts=[],[]
    
    for filename in flist:
    
        with h5py.File(filename, 'r') as f:

            print('Collecting data from',filename)
            
            dset            = f['protons']
            dset_columns    = f['columns']
            dset_selections = f['selections']
            dset_counts     = f['event_counts']
            
            #read the data
            columns = list( dset_columns )
            columns_str = [ item.decode("utf-8") for item in columns ]
            if chunk_size is None:
                start=[0]
                stop=[dset.shape[0]]
            else:
                entries = dset.shape[0]
                start = list( range( 0, entries, chunk_size ) )
                stop = start[1:] + [entries]
                
            for idx in range( len( start) ):
                print('\tCollecting events',start[idx], stop[idx] )

                df.append( pd.DataFrame( dset[start[idx]:stop[idx]], 
                                         columns=columns_str ) )
                df[-1]=df[-1][['Run', 'LumiSection', 'EventNum', 'CrossingAngle', 
                               'MultiRP', 'Arm', 'RPId1', 'RPId2', 'TrackX1', 'TrackY1', 'TrackX2', 'TrackY2',
                               'Xi', 'T', 'ThX', 'ThY', 'Time',
                               'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuPlus', 'XiMuMuMinus'] ].astype( { "Run": "int64", "LumiSection": "int64", "EventNum": "int64", "MultiRP": "int32", "Arm": "int32", "RPId1": "int32", "RPId2": "int32", "ExtraPfCands": "int32" } )
              
            #read the selection counters
            selections = list( dset_selections )
            selections_str = [ item.decode("utf-8") for item in selections ]        
            df_counts.append( pd.Series( list( dset_counts ), index=selections_str ) )
    
    n=len( df ) 
    print('\tReturning the result of %d merged datasets'%n)
    df_final=pd.concat(df)
    
    #merge the counts
    df_counts_final = df_counts[0]
    for idx in range( 1, len(df_counts) ):
        df_counts_final = df_counts_final.add( df_counts[idx] )

    #merge the data
    
    
    return df_final,df_counts_final
    
print('[Signal simulation]')
df_signal,df_counts_signal = GetData(PATH+'/output-MC2017-Elastic-Non3+3-PreSel.h5')
print('Selection counts')
print(df_counts_signal)


print('\n')
print('[Data (to be used as background)]')
data_files = [PATH+'/output-UL2017{}-PreSel-Rnd-Res20.h5'.format(era) for era in eras]
df_bkg,df_counts_bkg = GetData(data_files,chunk_size=1000000)
print('Selection counts')
print(df_counts_bkg)

## Prepare the data

We apply the following selection to the signal and data

* require $m_{ll}>110$ GeV
* set the dilepton xi according to the arm where a proton was reconstructed 
* use the proton reconstruction algorithm as required at the start of the notebook

In [ ]:
def PrepareData(df):
    
    """applies baseline selection cuts"""

    msk = ( df["InvMass"] >= 110. )

    msk1 = None
    msk2 = None
    if proton_selection == "SingleRP":
        # Single-RP in pixel stations
        msk1_arm = ( df["RPId1"] == 23 )
        msk2_arm = ( df["RPId1"] == 123 )
        multiRP=0
    elif proton_selection == "MultiRP":
        # Multi-RP
        msk1_arm = ( df["Arm"] == 0 )
        msk2_arm = ( df["Arm"] == 1 )
        multiRP=1
   
    df[ "XiMuMu" ] = np.nan
    df[ "XiMuMu" ].where( ~msk1_arm, df[ "XiMuMuPlus" ],  inplace=True )
    df[ "XiMuMu" ].where( ~msk2_arm, df[ "XiMuMuMinus" ], inplace=True )
    msk1 = msk & ( df["MultiRP"] == multiRP) & msk1_arm
    msk2 = msk & ( df["MultiRP"] == multiRP) & msk2_arm
   
    return df[msk1 | msk2].copy()

df_signal_prep = PrepareData(df_signal)
df_bkg_prep    = PrepareData(df_bkg)

In [ ]:
print('Signal prepared',df_signal_prep.shape)

train_vars=['Muon0Pt', 'Muon1Pt', 'InvMass',   'XiMuMu', 'Xi', 'Acopl','ExtraPfCands']

#draw the correlation matrix for the training variables
print(train_vars)
fig=plt.figure(figsize=(10, 10))
plt.matshow(df_signal_prep[train_vars].corr(), fignum=fig.number)
cb = plt.colorbar()
plt.clim(-1,1)
plt.title('Signal correlation Matrix', fontsize=16)
plt.show()

df_signal_prep[train_vars].head(10)

In [ ]:
print('Background prepared',df_bkg_prep.shape)

#draw the correlation matrix for these variables
print(train_vars)
fig=plt.figure(figsize=(10, 10))
plt.matshow(df_bkg_prep[train_vars].corr(), fignum=fig.number)
cb = plt.colorbar()
plt.clim(-1,1)
plt.title('Background correlation Matrix', fontsize=16)
plt.show()

df_bkg_prep[train_vars].head(10)

## Model building

Having the data prepared we now turn to building a simple model using the same training variables inspected above.
`sklearn` is used to define the training and test sample and to train a model which will do binary classification (0=background, 1=signal).
The model is based on a [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/tree.html#tree).
The [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) is then used to improve the base training of the DecisionTreeClassifier.

In [ ]:
X_sig=df_signal_prep[train_vars].copy()
X_bkg=df_bkg_prep[train_vars].copy()

y_sig = np.ones( len(X_sig) )
y_bkg = np.zeros( len(X_bkg) )

X = pd.concat( [X_sig, X_bkg] ) 
y = np.concatenate( [y_sig, y_bkg] )

In [ ]:
#divide into training an testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=True, random_state=42 )

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

#fit the parameters of the DecisionTreeClassifier
ada_clf = AdaBoostClassifier( DecisionTreeClassifier( max_depth=4 ),
                              n_estimators = 200,
                              algorithm="SAMME.R",
                              learning_rate = 0.5)
ada_clf.fit( X_train, y_train )
clf = ada_clf
print ( clf )

In [ ]:
#save the trained model
from joblib import dump, load
dump(clf, 'pps_longexercise_clf.joblib')

## Inspection of the training result

Check the accuracy of the predictions and the receiver-operating characteristic (ROC) curve. 
We use several function from sklearn to do this. 
In case you are not acquainted with them check the following links:
* [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) -  returns the fraction of correctly classified samples
* [roc_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) - used to compute the ROC curve for a binary classification
* [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) - computes the area under the ROC
You'll see that the result is extremely accurate and has a large background rejection power.

**TASK 1**

Can you identify which variables contribute mostly to this? (Hint: check for correlations with the classifier, re-train with N-1 variables are some possibilities.
Evaluating the impact via [permutation feature importance](https://scikit-learn.org/stable/modules/permutation_importance.html), will give you already a good hint.
If you are really into it try out a more complex (but also more correct) evaluation in this [example](https://colab.research.google.com/github/GilesStrong/lumin/blob/v0.5.1/examples/Feature_Selection.ipynb) using the [lumin](https://github.com/GilesStrong/lumin) package).

In [ ]:
from sklearn.metrics import accuracy_score
y_test_pred = clf.predict( X_test )
print('Accuracy on the test dataset is:',accuracy_score( y_test, y_test_pred ))

y_sig_pred = clf.predict( X_sig )
y_bkg_pred = clf.predict( X_bkg )
print ('Accuracy on the full signal data is', accuracy_score( y_sig, y_sig_pred ) )
print ('Accuracy on the full background data is', accuracy_score( y_bkg, y_bkg_pred ) )

In [ ]:
from sklearn import metrics

#get the probability for the event to be signal
y_test_proba = clf.predict_proba(X_test)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_test_proba)
auc = metrics.roc_auc_score(y_test, y_test_proba)

fig= plt.figure( figsize=(6,6) )
plt.plot(tpr,1/fpr,label="ROC AUC={:.3f}".format(auc))
plt.legend(loc='best')
plt.ylabel('Background rejection (1/false positive rate)')
plt.xlabel('Signal efficiency (true positive rate)')
plt.xlim(0.88,1.0)
plt.grid()
plt.show()

Plotting the discriminator shape. The distributions for the signal and background are characterized by a clear separation.

**TASK 2**

Can you make an analogous distribution, but for the probability for an event to be background? 
What is the relation between the two?

In [ ]:
fig = plt.figure( figsize=(8,8) )

bins=np.linspace(0.,1.,50)
plt.hist( y_test_proba[y_test == 0],  color='lightgray', bins=bins, label='Background')
plt.hist( y_test_proba[y_test == 1],  histtype='step',   bins=bins, label='Signal', linewidth=2 )
plt.xlabel('Probability')
plt.ylabel('Events (a.u)')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
def Plot2D(x,y,data_sig,data_bkg,msk_sig,msk_bkg,xran=[0.,0.15],yran=[0.,0.15]):
 
    """a simple routine to plot the signal and background components"""

    fig= plt.figure( figsize=(10,10) )
    plt.plot( data_bkg[x][ msk_bkg ], data_bkg[y][ msk_bkg ], 'ro', label='Background' )
    plt.plot( data_sig[x][ msk_sig ], data_sig[y][ msk_sig ], 'bo', label='Signal' )
    
    if 'Xi' in x and 'Xi' in y:
        plt.plot( xran,yran, 'k--', linewidth=1 )
        plt.plot( xran, [j*0.90 for j in yran], 'k:', linewidth=1 )
        plt.plot( xran, [j*1.10 for j in yran], 'k:', linewidth=1 )

    plt.xlim(*xran)
    plt.xlabel(x)
    plt.ylim(*yran)
    plt.ylabel(y)
    plt.legend(loc='best')

Test dataset: the events classified as signal correlate more strongly in the reconstucted $\xi$ plane. You can compare the result to the one obtained in the Data-Inspection notebook.

In [ ]:
msk_bkg = ( y_test_pred == 0 )
msk_sig = ( y_test_pred == 1 )
Plot2D('Xi','XiMuMu',X_test,X_test,msk_sig,msk_bkg)

Separate the correlations in the $\xi$ per PPS arm.
Check where the correct predictions in the signal and background datasets lie.

In [ ]:
msk_bkg_1 = None
msk_bkg_2 = None
msk_sig_1 = None
msk_sig_2 = None
if proton_selection == "SingleRP":
    msk_bkg_1 = ( df_bkg_prep[ "RPId1" ] == 23 ) & ( y_bkg_pred == 0 )
    msk_bkg_2 = ( df_bkg_prep[ "RPId1" ] == 123 ) & ( y_bkg_pred == 0 )
    msk_sig_1 = ( df_signal_prep[ "RPId1" ] == 23 ) & ( y_sig_pred == 1 )
    msk_sig_2 = ( df_signal_prep[ "RPId1" ] == 123 ) & ( y_sig_pred == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_1 = ( df_bkg_prep[ "Arm" ] == 0 ) & ( y_bkg_pred == 0 )
    msk_bkg_2 = ( df_bkg_prep[ "Arm" ] == 1 ) & ( y_bkg_pred == 0 )
    msk_sig_1 = ( df_signal_prep[ "Arm" ] == 0 ) & ( y_sig_pred == 1 )
    msk_sig_2 = ( df_signal_prep[ "Arm" ] == 1 ) & ( y_sig_pred == 1 )
Plot2D('Xi','XiMuMuPlus',df_signal_prep,df_bkg_prep,msk_sig_1,msk_bkg_1)
Plot2D('Xi','XiMuMuMinus',df_signal_prep,df_bkg_prep,msk_sig_2,msk_bkg_2)

Analogous to the previous set of plots but now we plot the background events which have been wrongly assigned as signal by the predictor.
False positive rate events are a handful events.

In [ ]:
msk_bkgerr_1 = None
msk_bkgerr_2 = None
if proton_selection == "SingleRP":
    msk_bkgerr_1 = ( df_bkg_prep[ "RPId1" ] == 23 ) & ( y_bkg_pred == 1 )
    msk_bkgerr_2 = ( df_bkg_prep[ "RPId1" ] == 123 ) & ( y_bkg_pred == 1 )
elif proton_selection == "MultiRP":
    msk_bkgerr_1 = ( df_bkg_prep[ "Arm" ] == 0 ) & ( y_bkg_pred == 1 )
    msk_bkgerr_2 = ( df_bkg_prep[ "Arm" ] == 1 ) & ( y_bkg_pred == 1 )
Plot2D('Xi','XiMuMuPlus', df_bkg_prep,df_bkg_prep, msk_bkgerr_1, msk_bkg_1)
Plot2D('Xi','XiMuMuMinus',df_bkg_prep,df_bkg_prep, msk_bkgerr_2, msk_bkg_2)

Finally the same distributions but now using events where the signal was misassigned as background.
Also an handful of events is obtained.

In [ ]:
msk_sigerr_1 = None
msk_sigerr_2 = None
if proton_selection == "SingleRP":
    msk_sigerr_1 = ( df_signal_prep[ "RPId1" ] == 23 ) & ( y_sig_pred == 0 )
    msk_sigerr_2 = ( df_signal_prep[ "RPId1" ] == 123 ) & ( y_sig_pred == 0 )
elif proton_selection == "MultiRP":
    msk_sigerr_1 = ( df_signal_prep[ "Arm" ] == 0 ) & ( y_sig_pred == 0 )
    msk_sigerr_2 = ( df_signal_prep[ "Arm" ] == 1 ) & ( y_sig_pred == 0 )
Plot2D('Xi','XiMuMuPlus', df_signal_prep,df_signal_prep, msk_sigerr_1, msk_sig_1)
Plot2D('Xi','XiMuMuMinus',df_signal_prep,df_signal_prep, msk_sigerr_2, msk_sig_2)

Compare the training variable distributions for events classified as signal or background in the test sample.

In [ ]:
def showDist(x,data,mask_sig, mask_bkg,nbins):
    
    """a simple function to compare signal-like and background-like"""
    
    fig = plt.figure( figsize=(6,6) )
    _,bins=np.histogram(data[x],bins=nbins)
    plt.hist( data[ x ][ mask_bkg ], color='lightgray', bins=bins, density=True, label='Background' )
    plt.hist( data[ x ][ mask_sig ], histtype='step',   bins=bins, density=True, label='Signal', linewidth=2)
    plt.xlabel(x)
    plt.ylabel('Events (a.u.)')
    plt.grid()
    plt.legend(loc='best')
    plt.show()
    
for x in train_vars:
    showDist(x,X_test,(y_test_pred==1),(y_test_pred==0),50)

## Analyze the data using the trained classifier

We now read, prepare and run the prediction for the data. 
The classifier output is used to further select the data and inspect some distributions.

In [ ]:
#load the data
print('\n')
print('[Data]')
data_files = [PATH+'/output-UL2017{}-PreSel.h5'.format(era) for era in eras]
df_data,df_counts_data = GetData(data_files,chunk_size=1000000)
print('Selection counts')
print(df_counts_data)

#prepare the data
df_data_prep=PrepareData(df_data)


#run the prediction
df_data_prep['clf_category'] = clf.predict( df_data_prep[train_vars] ).astype(int)
df_data_prep['clf_prob']     = clf.predict_proba( df_data_prep[train_vars] )[:,1]


df_data_prep.head()

Exciting! let's plot on the 2D $\xi$ planes the events which are classified as signal and background.
Looks like some candidate events are selected!

**TASK 2**

Discussion: are these really signal events or generated by false positive rate? 

In [ ]:
msk_bkg_1 = None
msk_bkg_2 = None
msk_sig_1 = None
msk_sig_2 = None
if proton_selection == "SingleRP":
    msk_bkg_1 = ( df_data_prep[ "RPId1" ] == 23 ) & ( df_data_prep['clf_category'] == 0 )
    msk_bkg_2 = ( df_data_prep[ "RPId1" ] == 123 ) & ( df_data_prep['clf_category'] == 0 )
    msk_sig_1 = ( df_data_prep[ "RPId1" ] == 23 ) & ( df_data_prep['clf_category'] == 1 )
    msk_sig_2 = ( df_data_prep[ "RPId1" ] == 123 ) & ( df_data_prep['clf_category'] == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_1 = ( df_data_prep[ "Arm" ] == 0 ) & ( df_data_prep['clf_category'] == 0 )
    msk_bkg_2 = ( df_data_prep[ "Arm" ] == 1 ) & ( df_data_prep['clf_category'] == 0 )
    msk_sig_1 = ( df_data_prep[ "Arm" ] == 0 ) & ( df_data_prep['clf_category'] == 1 )
    msk_sig_2 = ( df_data_prep[ "Arm" ] == 1 ) & ( df_data_prep['clf_category'] == 1 )

Plot2D('Xi','XiMuMuPlus', df_data_prep,df_data_prep,msk_sig_1,msk_bkg_1)
Plot2D('Xi','XiMuMuMinus',df_data_prep,df_data_prep,msk_sig_2,msk_bkg_2)

Let's use the catgorization to plot the distributions of the variables for events which are classified as signal or as background.
Maybe these distributions can give some insight into the characteristics of the events which are falling under the signal category.

In [ ]:
for x in train_vars:
    showDist(x,df_data_prep,(df_data_prep['clf_category']==1),(df_data_prep['clf_category']==0),50)

Finally, let's plot the events which have been classified as a signal against a background prediction,
based on the event-mixed data which has been used for the training of the MVA.

In [ ]:
def compareToBackgroundPrediction(var,data,msk_data,bkg,msk_bkg,bins=20,resample_factor=20):

    fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

    #define the binning if only nbins were given
    if isinstance(bins,int):
        _,bins=np.histogram(df_data_prep[var],bins=bins)

    counts={}
    errors={}
    for arm in [0,1]:
      
        counts[arm], bin_edges = np.histogram( data[var][msk_data[arm]], bins=bins )
        errors[arm] = np.sqrt( counts[arm] )
        bin_centres = ( bin_edges[:-1] + bin_edges[1:] ) / 2.
        axes[arm].errorbar(bin_centres, counts[arm], yerr=errors[arm], fmt='o', label='Data')

        weights = None
        if resample_factor > 1:
            weights = np.full_like( bkg[ "Xi" ][ msk_bkg[arm] ], ( 1./resample_factor ) )
    
        axes[arm].hist( bkg[ var ][ msk_bkg[arm] ], bins=bins, weights=weights, label='background' )
        
    #final tweak to display both y-axis with the same range
    idx_ymax = np.argmax( np.concatenate( [counts[0], counts[1]] ) )
    y_max = np.concatenate( [counts[0], counts[1]] )[idx_ymax] + 2*np.concatenate( [errors[0], errors[1]] )[idx_ymax]

    for arm in [0,1]:    
        axes[arm].set_ylim( top=y_max )
        axes[arm].set_ylabel('Events')
        axes[arm].set_xlabel(var)
        axes[arm].grid()

    plt.show()
        
msk_bkg_pred_1 = None
msk_bkg_pred_2 = None
if proton_selection == "SingleRP":
    msk_bkg_pred_1 = ( df_bkg_prep[ "RPId1" ] == 23 ) & ( y_bkg_pred == 1 )
    msk_bkg_pred_2 = ( df_bkg_prep[ "RPId1" ] == 123 ) & ( y_bkg_pred == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_pred_1 = ( df_bkg_prep[ "Arm" ] == 0 ) & ( y_bkg_pred == 1 )
    msk_bkg_pred_2 = ( df_bkg_prep[ "Arm" ] == 1 ) & ( y_bkg_pred == 1 )

msk_data=[msk_sig_1,msk_sig_2]
msk_bkg=[msk_bkg_pred_1,msk_bkg_pred_2]
for var in train_vars:
    compareToBackgroundPrediction(var,df_data_prep,msk_data,df_bkg_prep,msk_bkg)

**TASK 3**

Add the DeltaXi variable to the data and background data frames and plot it in the same style as before. 
What are your conclusions?

In [ ]:
#write your piece of code here

#stuck? uncomment the following piece of code and run to see the solution
#%load snippets/ClassificationTask3.py

**TASK 4**

Once you are happy with the features, model and training, save your model so that it can be used for the statistical analysis.